# API Demo: Search, Order, and Download Capella's Rotterdam Data Set

In [ ]:
# Capella has collected two AOIs - 10 x 5 km each. Each AOI will be covered from 8am to 8pm over two days, 
# 6 times each day to simulate roughly two hour revisit at very high resolution. 
# The main features in the AOIs are: the port, ships (tankers and commercials), shipping containers, cars,
# parking lot, stadium, train rails, floating oil tanks, downtown, vegetated areas. 
# Imagery is be generated using backprojection in map coordinates.

In [ ]:
# Required libraries:
# requests
# json
# urllib

In [ ]:
# Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.
# {"username": "yourusername","password": "xxxxxxxxx"}

In [ ]:
import requests
import json

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

# Capella PROD API endpoints
URL = 'https://api.data.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/stac/search'
orders = '/orders/'

# Load my username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']


#print(username)


In [ ]:
# Get a valid token from the auth service
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
print(r.json())
accesstoken = r.json()["accessToken"]

# Print the tokens
print("Access Token: " + accesstoken)

headers = {'Authorization':'Bearer ' + accesstoken}
#print(headers)

In [ ]:
# See what collections are available
r = requests.get(URL + collections, headers=headers)

# Print results
#print(URL + collections)
p(r.json())


In [ ]:
# Posting some simple search filters specifically for Rotterdam data set 
filters = {
  "bbox": [4.968939208984375,51.04426252720166,4.033856201171875,52.560346153985355],
  "time": "2019-08-01T00:00:00Z/2019-12-31T12:31:12Z",
  "limit": 500, # 
  "sort": [{"field": "dtr:start_datetime"}],
}
headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)

# Inspect the results
p(r.json())

In [ ]:
# Make an Order
features = r.json()["features"]
granulelist = []
#p(features)
# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)

myorder = {"Items": granulelist}
#print(myorder)

# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
p(r.json())

In [ ]:
# Get the STAC records with the signed URLs using the /download endpoint
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
p(r.json())


In [ ]:
# Download the results
features = r.json()
#p(features)

basefp = 'C:/data/' # Local directory to save data

for feature in features:

    filepath = feature["assets"]["HH"]["href"]
    #print(filepath)

    filename = filepath[filepath.rfind("/")+1:]
    #print(filename)
    sep = "?"
    truncname = filename.split(sep, 1)[0]
    #print(truncname)
    
    outfp = basefp + truncname
    
    import urllib
    f = urllib.request.urlretrieve(filepath, outfp)
